<a href="https://colab.research.google.com/github/NotARectangle/Honours2021/blob/main/usingTrainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 2.0MB 8.2MB/s 
     |████████████████████████████████| 890kB 48.4MB/s 
     |████████████████████████████████| 3.2MB 51.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=dab85c2a4466da3ba5b40e2a0fdc84af310662c8a87e62722c0ac38d63a68ebd
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


Persona ID file

In [2]:

def setSpecTokens(model, tokenizer):
    tokenizer.add_special_tokens({"bos_token": "<bos>", "eos_token": "<eos>", "pad_token": "<pad>"})#, "additional_special_tokens" : ["PICARD:"]})
    print(tokenizer.bos_token)
    print(tokenizer.eos_token)
    # adapt model to changes from tokenizer
    model.resize_token_embeddings(len(tokenizer))


def prepare_inputs(persona, history, reply, model, tokenizer):

    # replace EOS token for PICARD:
   # reply = [reply[0].replace("<eos>", " PICARD:")]
    max_input = 1024 #GPT2 max input sequence length.
    string_input = persona + history + reply;

    #print(string_input)
    #Get scene speakers
    speaker_token_re = "([A-Z]+:)"
    speakers = []
    repeat = False
    for input in string_input:
        if re.match(speaker_token_re, input):
            speaker = re.findall(speaker_token_re, input)
            for s in speaker:
                if s not in speakers and s not in tokenizer.additional_special_tokens:
                    speakers = speakers + speaker

   # print(tokenizer.additional_special_tokens)
   
    #encode all inputs
    sequence = [tokenizer.encode(s) for s in string_input]



    #currentSpeaker = spek_token_ids[0] # start with selected character
    #currentSpeaker = 0
    token_type_ids = []
    words = []

    for seq in sequence:
        type = 0
        if seq == sequence[(len(sequence)-1)]:
            type = 1
        for token in seq:
            #cocacennate all tokens in sequence together
            words.append(token)

            token_type_ids.append(type)
            """
            if token in spek_token_ids:
                currentSpeaker = spek_token_ids[spek_token_ids.index(token)]
            if token is tokenizer.bos_token_id or token is tokenizer.eos_token_id:
                token_type_ids.append(token)
            else:
                token_type_ids.append(currentSpeaker)
            """
    #check if inputs are to long.

    positions = list(range(len(words)))

    return words, sequence, positions, token_type_ids


Dataloader file

In [3]:
# load dataset
def load_dataset(filePath):
    data = json.load(open(filePath, 'r', encoding="utf-8"))

    return data

def seperate_train_test(data):
    personas = data.keys()
    dict = {"Train" : {}, "Test": {}}
    train_utt = []
    test_utt = []
    for persona in personas:
        utter = data[persona]["utterances"]
        u_train, u_test = train_test_split(utter, test_size=0.20)
        dict["Train"][persona] = {"PersonaID": data[persona]["PersonaID"], "utterances": u_train}
        dict["Test"][persona] = {"PersonaID": data[persona]["PersonaID"], "utterances": u_train}
    return dict

def prepare_inputs_from_data(data, model, tokenizer):
    input_dict = {"input_ids": [], "token_type_ids":[]
        , "labels": []}
    for key in data:
      persona = data[key]["PersonaID"]
      utterances = data[key]["utterances"]
      index = 0
  #    while index < len(utterances):
      trainingLen = len(utterances) - int(len(utterances) * 0.25)
      #trainingLen = 15000
      while index < trainingLen: #less to make it faster for testing
          history = utterances[index]["history"]
          reply = utterances[index]["reply"]
          #tokenize and build word sequence sing prepare inputs
          words, sequence, positions, token_type_ids = prepare_inputs(persona, history, reply, model, tokenizer)
          if len(words) < 1000:
              #Add inputs to input_dict
              input_dict["input_ids"].append(words)
              last_token = len(words)-1
          #   input_dict["mc_token_ids"].append(last_token)
              labels = []

              in_reply = False
              for seq in sequence:
                  # make labels pointing to reply
                  j = 0
                  label = -100
                  # check if reply
                  if seq == sequence[(len(sequence)-1)]:
                      #label = 10
                      in_reply = True
                  while j < len(seq):
                      if in_reply is True:
                          label = seq[j]
                      labels.append(label)
                      j += 1

            # input_dict["lm_targets"].append(lm_targets)
          #   input_dict["positions"].append(positions)
              input_dict["token_type_ids"].append(token_type_ids)
              input_dict["labels"].append(words)
          index += 1
          #print(index)

    print("finished while prep input")
    """
    #pad and then convert to tensors.
    for key in input_dict:
        if key != "mc_token_ids" and key != "attention_mask":
            paddedInput = padding(input_dict[key], tokenizer)

    count = 0
    while count < len(input_dict["labels"]):
        newLabels = []
        for i in input_dict["labels"][count]:
            if i == tokenizer.pad_token_id:
                newLabels.append(-100)
            else:
                newLabels.append(i)
        input_dict["labels"][count] = newLabels
        count+=1
    
    # add attention mask
    att_Mask = []
    for input in input_dict["input_ids"]:
        item = []
        for i in input:
            if i != tokenizer.pad_token_id:
                item.append(1)
            else:
                item.append(0)
        att_Mask.append(item)

    input_dict["attention_mask"] = att_Mask
    print(input_dict["attention_mask"][2])
    print(input_dict["attention_mask"][1])
    """
    return input_dict

 #build tensor dataset.

# split dataset in train and test dataset

def convert_to_tensors(input_dict, pad_value):
    tensor_dataset = {}
    length = []
    pad_v = pad_value
    for item in input_dict["input_ids"]:
        length.append(len(item))
    print(length[0])

    for key in input_dict:
        unpad_tensors = []
        for item in input_dict[key]:
            tensor = torch.tensor(item)
            unpad_tensors.append(tensor)
        #pad tensor
        #if key equals labels pad -100 to ignore for calculating loss
        if key == "labels":
            pad_v=-100
        else:
            pad_v=pad_value
        tensors = pad_sequence(unpad_tensors, batch_first=True, padding_value=pad_v)
        tensor_dataset[key] = tensors

    return tensor_dataset



Training file

In [4]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import AdamW, AutoModel, Trainer, TrainingArguments

class tng_dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings["input_ids"])


def train(input_dict, model):
    print("start training method")
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
   # device = torch.device("cpu")
    model.to(device)
    model.train()
    optim = AdamW(model.parameters(), lr=5e-5)

  #  train_dataset = TensorDataset(*input_dict["train"])
  #  test_dataset = TensorDataset(*input_dict["test"])
    train_dataset = tng_dataset(input_dict["train"])
    test_dataset = tng_dataset(input_dict["test"])
    #convert to dataset.
    train_loader = DataLoader(train_dataset, batch_size=10)

    training_args = TrainingArguments(
        output_dir='./results',  # output directory
        num_train_epochs=3,  # total number of training epochs
        per_device_train_batch_size=9,  # batch size per device during training
        per_device_eval_batch_size=10,  # batch size for evaluation
        warmup_steps=500,  # number of warmup steps for learning rate scheduler
        weight_decay=0.01,  # strength of weight decay
        logging_dir='./logs',  # directory for storing logs
        logging_steps=10,
    )

    trainer = Trainer(
        model=model,  # the instantiated 🤗 Transformers model to be trained
        args=training_args,  # training arguments, defined above
        train_dataset=train_dataset,  # training dataset
        eval_dataset=test_dataset  # evaluation dataset
    )

    trainer.train()
    trainer.save_model("drive/MyDrive/Honours2021/TNGv3")


def config_dir():
    return "/MakeItSo"

Main File from model folder

In [5]:
from transformers import GPT2Tokenizer, pipeline, GPT2LMHeadModel
import json
import re
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
setSpecTokens(model, tokenizer)

filepath = 'drive/MyDrive/Honours2021/tngPersonaData.json'
data = load_dataset(filepath)

newDict = seperate_train_test(data)
#make train and test dict with all characters

train_dict = newDict["Train"]
test_dict = newDict["Test"]
input_dict_train = prepare_inputs_from_data(train_dict, model, tokenizer)
input_dict_test = prepare_inputs_from_data(test_dict, model, tokenizer)
pad_value = tokenizer.pad_token_id
train_tensor_dataset = convert_to_tensors(input_dict_train, pad_value)
test_tensor_dataset = convert_to_tensors(input_dict_test, pad_value)




<bos>
<eos>
finished while prep input
finished while prep input
203
203


In [6]:
tensor_dataset = {"train" : train_tensor_dataset, "test" : test_tensor_dataset}

train(tensor_dataset, model)

start training method


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


Step,Training Loss
10,47.313400
20,30.635700
30,19.481600
40,12.313300
50,9.237500
60,7.653800
70,6.688300
80,5.825700
90,5.089800
100,4.512400


Step,Training Loss
10,47.313400
20,30.635700
30,19.481600
40,12.313300
50,9.237500
60,7.653800
70,6.688300
80,5.825700
90,5.089800
100,4.512400


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


In [7]:
tokenizer.save_pretrained("drive/MyDrive/Honours2021/TNGv3")

('drive/MyDrive/Honours2021/TNGv3/tokenizer_config.json',
 'drive/MyDrive/Honours2021/TNGv3/special_tokens_map.json',
 'drive/MyDrive/Honours2021/TNGv3/vocab.json',
 'drive/MyDrive/Honours2021/TNGv3/merges.txt',
 'drive/MyDrive/Honours2021/TNGv3/added_tokens.json')

In [8]:
modelPath = "drive/MyDrive/Honours2021/TNGv3"
makeItSo = pipeline('text-generation',model=modelPath, tokenizer=modelPath,config={'max_length':1200})

In [9]:
makeItSo("USER: What do you think of this? PICARD: ")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "USER: What do you think of this? PICARD: COMPUTER: I am Commander William T Riker of the Enterprise.RIKER: I'm not a mind control expert. I came here because I sensed you were in pain"}]

In [15]:
makeItSo("TROI: How are you today captain? PICARD:")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "TROI: How are you today captain? PICARD: I'm Commander William T Riker.PICARD: I am Jean-Luc Picard, of the Federation Starship, USSEnterprise.PICARD: I'm not a"}]

In [16]:
makeItSo("TROI: Good day Captain. PICARD:")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'TROI: Good day Captain. PICARD: I am Jean-Luc Picard, Captain of the Enterprise. We come in peace.PICARD: I am Captain Picard.PICARD: I am Jean-Luc Picard, of'}]